In [ ]:
%pip uninstall nomad -y
%pip install git+https://github.com/Watts-Lab/nomad.git@data-gen

In [1]:
import pandas as pd
import numpy as np
import numpy.random as npr
from datetime import datetime, timedelta
import calendar
from pyproj import Transformer
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures
import multiprocessing
from functools import partial
import copy

import nomad.city_gen as cg
from nomad.city_gen import City, Building, Street
import nomad.traj_gen
from nomad.traj_gen import Agent, Population
from nomad.constants import DEFAULT_SPEEDS, FAST_SPEEDS, SLOW_SPEEDS, DEFAULT_STILL_PROBS
from nomad.constants import FAST_STILL_PROBS, SLOW_STILL_PROBS, ALLOWED_BUILDINGS

import os
os.environ['TZ'] = 'UTC'

### Generate N agents

In [2]:
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)


def garden_city_to_lat_long(agent, 
                            sparse_traj=True, 
                            full_traj=True):
    if sparse_traj:
        df = agent.sparse_traj
        df['x'] = 15*df['x'] + 4265699
        df['y'] = 15*df['y'] - 4392976

        df['latitude'], df['longitude'] = transformer.transform(
            df['x'].values, df['y'].values)

        df = df[['identifier', 'unix_timestamp', 'latitude', 'longitude']]
        df = df.rename(columns={'identifier': 'uid', 'unix_timestamp': 'timestamp'})
        df = df.reset_index(drop=True)

        agent.sparse_traj = df

    if full_traj:
        df = agent.trajectory
        df['x'] = 15*df['x'] + 4265699
        df['y'] = 15*df['y'] - 4392976

        df['latitude'], df['longitude'] = transformer.transform(
            df['x'].values, df['y'].values)

        df = df[['identifier', 'unix_timestamp', 'latitude', 'longitude']]
        df = df.rename(columns={'identifier': 'uid', 'unix_timestamp': 'timestamp'})
        df = df.reset_index(drop=True)

        agent.trajectory = df
    return None

In [6]:
%%time

N = 1
npr.seed(100)

city = cg.load('garden-city.pkl')
population_n = Population(city)
population_n.generate_agents(N,
                             start_time=datetime(2024, 1, 1, hour=8, minute=0),
                             dt=1, seed=123)

def generate_agent_trajectory(agent_id, agent, seed):
    
    # May want to put more thought into the ranges
    beta_start = npr.uniform(60, 1200)
    beta_duration = npr.uniform(15, 180)
    beta_ping = npr.uniform(5, 30)
    
    param = (beta_start, beta_duration, beta_ping)
    
    for month in range(1,3):
        days = calendar.monthrange(2024, month)[1]
        population_n.generate_trajectory(agent, 
                                         T=datetime(2024, month, days, hour=23, minute=59), 
                                         seed=seed)
    
        agent.sample_traj_hier_nhpp(*param, 
                                    seed=seed,
                                    reset_traj=True)
    
    garden_city_to_lat_long(agent,
                        sparse_traj=True,
                        full_traj=False)
    
    return agent_id, copy.deepcopy(agent)

manager = multiprocessing.Manager()
shared_roster = manager.dict(population_n.roster)

with ProcessPoolExecutor() as executor:
    futures = [
        executor.submit(generate_agent_trajectory, agent_id, agent, i)
        for i, (agent_id, agent) in enumerate(shared_roster.items())
    ]
    for future in futures:
        agent_id, agent = future.result()
        shared_roster[agent_id] = agent

population_n.roster = dict(shared_roster)

population_n.save_pop(bucket='synthetic-raw-data', prefix='1mil-agents/')

CPU times: user 604 ms, sys: 174 ms, total: 778 ms
Wall time: 21.3 s


In [8]:
tom=population_n.roster['wizardly_joliot']
df = tom.sparse_traj
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')

df

,uid,timestamp,latitude,longitude,datetime
0,wizardly_joliot,1704114660,38.320899,-36.667093,2024-01-01 13:11:00
1,wizardly_joliot,1704115680,38.320879,-36.667166,2024-01-01 13:28:00
2,wizardly_joliot,1704116100,38.320977,-36.667390,2024-01-01 13:35:00
3,wizardly_joliot,1704116640,38.320909,-36.667196,2024-01-01 13:44:00
4,wizardly_joliot,1704119640,38.321728,-36.667468,2024-01-01 14:34:00
...,...,...,...,...,...
670,wizardly_joliot,1709201400,38.320187,-36.667625,2024-02-29 10:10:00
671,wizardly_joliot,1709202600,38.320120,-36.667609,2024-02-29 10:30:00
672,wizardly_joliot,1709202720,38.320081,-36.667579,2024-02-29 10:32:00
673,wizardly_joliot,1709203740,38.320209,-36.667560,2024-02-29 10:49:00


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(5, 5))

ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])

ax.scatter(x=tom.sparse_traj.latitude, 
           y=tom.sparse_traj.longitude, 
           s=0.5, color='red', alpha=0.3)